In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import os

def get_block_group_population(census_gdf, census_api_key, year, output_file, csv_output_file, aggregate_file):
    """
    Retrieves population data for block groups based on a given year and saves the processed data to a file.

    Parameters:
        census_gdf (GeoDataFrame): The GeoDataFrame containing the geometries for the census block groups.
        census_api_key (str): The Census API key.
        year (int): The year for which the data is retrieved (e.g., 2013, 2017, 2022).
        output_file (str): The filename where the processed data will be saved.
        csv_output_file (str): The filename for saving the processed CSV data.
        aggregate_file (str): The filename for appending population density data.

    Returns:
        GeoDataFrame: The processed GeoDataFrame with population and PDI data.
    """
    # Check if the API key is provided
    if not census_api_key:
        raise ValueError("API key must be provided as the 'census_api_key' parameter.")

    # Get unique state and county FIPS codes from the GeoDataFrame
    state_fips = census_gdf.STATEFP.unique()[0]
    county_fips = census_gdf.COUNTYFP.unique()

    census_pop = []

    # Retrieve census data by block group for the specified state and county
    for county_fip in county_fips:
        params = {
            'get': 'B01003_001E',
            'for': 'block group:*',
            'in': f'state:{state_fips} county:{county_fip}',
            'key': census_api_key
        }
        url = f'https://api.census.gov/data/{year}/acs/acs5'
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            columns = data[0]
            values = data[1:]
            df = pd.DataFrame(values, columns=columns)
            census_pop.append(df)
        else:
            print(f"Error fetching data for county {county_fip}: {response.text}")
            continue

    # Concatenate all dataframes
    if census_pop:
        census_pop_df = pd.concat(census_pop, ignore_index=True)
    else:
        print("No data retrieved from the Census API.")
        return None

    print(f"Data retrieved from the Census API (first few rows):\n{census_pop_df}")

    # Rename columns in census_gdf to match the names in census_pop_df
    census_gdf.rename(columns={
        'STATEFP': 'state',
        'COUNTYFP': 'county',
        'TRACTCE': 'tract',
        'BLKGRPCE': 'block group',
    }, inplace=True)

    # Ensure that 'state', 'county', 'tract', 'block group' are strings and zero-padded as necessary
    census_gdf['state'] = census_gdf['state'].astype(str).str.zfill(2)
    census_gdf['county'] = census_gdf['county'].astype(str).str.zfill(3)
    census_gdf['tract'] = census_gdf['tract'].astype(str).str.zfill(6)
    census_gdf['block group'] = census_gdf['block group'].astype(str).str.zfill(1)  # Block group is typically one digit

    # Similarly for census_pop_df
    census_pop_df['state'] = census_pop_df['state'].astype(str).str.zfill(2)
    census_pop_df['county'] = census_pop_df['county'].astype(str).str.zfill(3)
    census_pop_df['tract'] = census_pop_df['tract'].astype(str).str.zfill(6)
    census_pop_df['block group'] = census_pop_df['block group'].astype(str).str.zfill(1)
    census_pop_df['B01003_001E'] = pd.to_numeric(census_pop_df['B01003_001E'], errors='coerce')

    # Merge census_gdf with the population DataFrame on 'state', 'county', 'tract', 'block group'
    merged_gdf = census_gdf.merge(census_pop_df[['state', 'county', 'tract', 'block group', 'B01003_001E']], on=['state', 'county', 'tract', 'block group'], how='left')

    # Continue with the rest of the code
    # Rename the population column to something more user-friendly
    merged_gdf.rename(columns={'B01003_001E': 'Population Count'}, inplace=True)

    # Create the 'GEOID' column
    merged_gdf['GEOID'] = merged_gdf['state'] + merged_gdf['county'] + merged_gdf['tract'] + merged_gdf['block group']

    # Calculate area in square kilometers
    area_sqkm = merged_gdf['ALAND'] / 10**6
    merged_gdf['Polygon Area'] = area_sqkm

    # Calculate population density
    merged_gdf['Population Density'] = merged_gdf['Population Count'] / area_sqkm

    # Calculate PDI (Population Density Index) as a percentage of the max density
    max_density = merged_gdf['Population Density'].max() if not merged_gdf['Population Density'].isna().all() else 1
    merged_gdf['PDI'] = (merged_gdf['Population Density'] / max_density) * 100  # PDI as percentage of max, ranging from 0 to 100

    # Ensure that merged_gdf has a valid geometry
    merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry')

    # Calculate centroids and extract coordinates
    merged_gdf['Centroid'] = merged_gdf.geometry.centroid
    merged_gdf['Coordinates'] = merged_gdf['Centroid'].apply(lambda x: f"({x.y:.6f}, {x.x:.6f})")

    # Drop the Centroid column before saving to GeoJSON
    merged_gdf = merged_gdf.drop(columns=['Centroid'])

    # Save processed data to GeoJSON
    merged_gdf.to_file(output_file, driver="GeoJSON")
    print(f"Data saved to {output_file}")

    # Prepare CSV output with specified column names and order
    csv_output = merged_gdf[['GEOID', 'Population Count', 'Population Density', 'PDI', 'Polygon Area', 'Coordinates']]
    csv_output.to_csv(csv_output_file, index=False)
    print(f"CSV data saved to {csv_output_file}")
    
    # Append population density to aggregate file
    geoidandyear = merged_gdf["GEOID"] + "+" + str(year)
    aggregate_df = pd.DataFrame({
        "GEOID": geoidandyear,
        "Population Density": merged_gdf["Population Density"]
    })
    if os.path.exists(aggregate_file):
        aggregate_df.to_csv(aggregate_file, mode='a', header=False, index=False)
    else:
        aggregate_df.to_csv(aggregate_file, mode='w', header=True, index=False)

    return merged_gdf

# Block Group execution

census_api_key = "bb8ddb8b99dc18f4759d67d905c25e1486077c4d"  # Replace with your actual Census API key

# Ensure you have year-specific GeoJSON files for block groups
census_gdf_2013 = gpd.read_file('block_groups.geojson')
census_gdf_2013 = get_block_group_population(
    census_gdf_2013, year=2013, census_api_key=census_api_key,
    output_file="block_groups_2013_PDI.geojson",
    csv_output_file="block_groups_2013_PDI.csv",
    aggregate_file="PDI_bg_all.csv"
)

census_gdf_2017 = gpd.read_file('block_groups.geojson')
census_gdf_2017 = get_block_group_population(
    census_gdf_2017, year=2017, census_api_key=census_api_key,
    output_file="block_groups_2017_PDI.geojson",
    csv_output_file="block_groups_2017_PDI.csv",
    aggregate_file="PDI_bg_all.csv"
)

census_gdf_2022 = gpd.read_file('block_groups.geojson')
census_gdf_2022 = get_block_group_population(
    census_gdf_2022, year=2022, census_api_key=census_api_key,
    output_file="block_groups_2022_PDI.geojson",
    csv_output_file="block_groups_2022_PDI.csv",
    aggregate_file="PDI_bg_all.csv"
)
